In [ ]:
%%capture
!pip install --upgrade transformers

In [ ]:
%%capture
!pip install datasets

In [ ]:
%%capture
!pip install accelerate -U

In [ ]:
%%capture
!pip install evaluate

In [ ]:
%%capture
!pip install seqeval

In [ ]:
%%capture
!pip install scikit-learn

In [ ]:
%%capture
!pip install matplotlib

In [ ]:
%%capture
!pip install sklearn

In [ ]:
from google.colab import drive
import torch
import pandas as pd
import evaluate
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tokenizers import Encoding
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForTokenClassification, AdamW, get_linear_schedule_with_warmup, Trainer, TrainingArguments,  BatchEncoding

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda")
json_data = '/content/drive/MyDrive/davodi.json'
df = pd.read_json(json_data)
df = df.drop(['_id', 'sentence', 'frame', 'lexicalUnit', 'status', 'issuer', 'is_active', 'createdAt', 'updatedAt', 'PId', 'lang', 'description', 'lexicalUnitHint', 'reviewer', 'lexicalUnitHelper', 'frameHelper', 'frameName', 'lexicalUnitName'], axis = 1)

In [ ]:
def framenet_tags(FN_tags):
    framenet_pattern = []
    for FN_tag in FN_tags:
        tag_type = FN_tag.get('tagType')
        if tag_type == 5:
            element = FN_tag.get('element', {})
            element_name = element.get('name', '')
            framenet_pattern.append(element_name)
        else:
            framenet_pattern.append('O')
    return framenet_pattern

In [ ]:
df['frameNetTags'] = df['frameNetTags'].apply(lambda FN_tags : framenet_tags(FN_tags))

In [ ]:
df

,words,frameNetTags,propBankTags
0,خانم نزهت‌الدوله گرچه تا به حال سه تا شوهر کرد...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,خانم نزهت‌الدوله گرچه تا به حال سه تا شوهر کرد...,"[جستجو_کننده, جستجو_کننده, O, O, O, O, O, O, O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,تو به من مجال دادی تا پروانه شوم .,"[مرجع_ذیصلاح, قهرمان, قهرمان, O, O, عمل, عمل, ...","[B-A0, B-A2, O, B-NVE, B-V, B-PRP, O, O, O]"
3,تو به من مجال دادی تا پروانه شوم .,"[O, O, O, O, O, O, دسته_نهایی, O, O]","[O, O, O, O, O, O, B-A2, B-V, O]"
4,هر زمان این غریزه و فطرت و رشد سنی و عقلی و اج...,"[O, O, موجودیت, موجودیت, موجودیت, موجودیت, موج...","[O, O, O, B-A2, O, O, O, O, O, O, O, O, O, B-N..."
...,...,...,...
32724,روسیه به زودی برای عمل به تعهدهای خود به اروپا...,"[بحث‌کنندگان, زمان, زمان, مسئله, مسئله, مسئله,...","[B-A0, B-TMP, O, B-A2, O, O, O, O, O, O, O, O,..."
32725,منازعه و مشاجرۀ پدر و مادر اقدام ناخوشایندی اس...,"[O, O, O, بحث‌کنندگان, بحث‌کنندگان, بحث‌کنندگا...","[B-N-V, O, O, B-A0, B-A1, O, O, O, O, O, O, O,..."
32726,منازعه و مشاجرۀ پدر و مادر اقدام ناخوشایندی اس...,"[روش, روش, روش, عامل, عامل, عامل, O, توضیح, O,...","[B-A1, O, O, O, O, O, B-A2, O, B-V, O, O, O, O..."
32727,منازعه و مشاجرۀ پدر و مادر اقدام ناخوشایندی اس...,"[O, O, O, O, O, O, O, O, O, O, شاخص, شاخص, O, ...","[O, O, O, O, O, O, O, O, O, O, B-Arg2_B1, O, B..."


In [ ]:
train_data, temp_data = train_test_split(df, test_size=0.2, random_state=42)
validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)
print("Training set size:", len(train_data))
print("Validation set size:", len(validation_data))
print("Test set size:", len(test_data))

In [ ]:
df["Words"] = df["words"].apply(lambda sentence: [word.strip() for word in sentence.split(' ')])

<ipython-input-20-ce01e66bb15d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Words"] = df["words"].apply(lambda sentence: [word.strip() for word in sentence.split(' ')])


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("myrkur/Persian-ModernBert-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.30M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [ ]:
tokenized_input = tokenizer(df['words'][0].split(' '), is_split_into_words=True)
tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

['خانم',
 'نز',
 'هت',
 '\u200cال',
 'دوله',
 'گرچه',
 'تا',
 'به',
 'حال',
 'سه',
 'تا',
 'شوهر',
 'کرده',
 'و',
 'شش',
 'بار',
 'زاییده',
 'و',
 'دو',
 'تا',
 'از',
 'دختر',
 'هایش',
 'هم',
 'به',
 'خان',
 '[UNK]',
 'داماد',
 'فرستاده',
 '_',
 'شده\u200cاند',
 'و',
 'حالا',
 'دیگر',
 'برای',
 'خودش',
 'مادربزرگ',
 'شده',
 '_',
 'است',
 '،',
 'باز',
 'هم',
 'عقیده',
 'دارد',
 'که',
 'پیری',
 'و',
 'جوانی',
 'دست',
 'خود',
 'آدم',
 'است',
 'و',
 'گرچه',
 'سر',
 'و',
 'همسر',
 'و',
 'خوی',
 'شان',
 'و',
 'دوستان',
 'می\u200cگویند',
 'که',
 'پنجاه',
 'سالی',
 'دارد',
 'ولی',
 'هنوز',
 'دو',
 'دستی',
 'به',
 'جوانی',
 '\u200cاش',
 'چسبیده',
 'و',
 'هنوز',
 'هم',
 'در',
 'جستجوی',
 'شوهر',
 'ایده\u200cآل',
 'به',
 'این',
 'در',
 'و',
 'آن',
 'در',
 'می\u200cزند',
 '.']

In [ ]:
def create_role_id_dictionary():
    """Creates a dictionary that maps PropBank roles to numerical IDs."""
    #roles = set()
    frame_elements = set()
    for _, row in df.iterrows():
    #    prop_str = row['propBankTags']
        framenet_str = row['frameNetTags']
    #    roles.update(prop_str)
        frame_elements.update(framenet_str)
    #role_id_dict = defaultdict(int)
    frame_id_dict = defaultdict(int)
    index = 1
    #for role in roles:
    #    role_id_dict[role] = index
    #    index += 1
    for frame_element in frame_elements:
        frame_id_dict[frame_element] = index
        index += 1
    #return role_id_dict, frame_id_dict, roles, frame_elements
    return frame_id_dict, frame_elements

In [ ]:
frame2id, frame_elements = create_role_id_dictionary()
print("Frame2ID:" + str(frame2id))
print("Frame Elements:" + str(frame_elements))

Frame2ID:defaultdict(<class 'int'>, {'هدف': 1, 'هدف_رویداد': 2, 'طرف_متأثر': 3, 'محدوده': 4, 'کنش': 5, 'کالا': 6, 'مالکیت': 7, 'گیرنده': 8, 'مسیر': 9, 'توصیفگر': 10, 'توصیفی': 11, 'قهرمان': 12, 'رویداد_همزمان': 13, 'تشخیص\u200cگر': 14, 'مقصد': 15, 'مبدأ': 16, 'کاربرد': 17, 'شمارش': 18, 'مسافر': 19, 'رقابت': 20, 'عمل': 21, 'کنشگر': 22, 'متن': 23, 'محل_اجرا': 24, 'بارش': 25, 'موجودیت_جستجوشده': 26, 'مورد_1': 27, 'نام': 28, 'رویداد_مقایسه': 29, 'عقیده': 30, 'فرستنده': 31, 'پرداخت_کننده': 32, 'جریمه': 33, 'مجری2': 34, 'کنش\u200cپذیر': 35, 'مقوله': 36, 'رویداد_پیش\u200cزمینه': 37, 'توزیع\u200cکننده': 38, 'درجه': 39, 'تاثیر': 40, 'وسیله': 41, 'موجود': 42, 'محل': 43, 'ابزار': 44, 'رویداد': 45, 'پذیرنده': 46, 'بسامد': 47, 'کیفیت_نهایی': 48, 'مورد': 49, 'توصیف': 50, 'پیکر': 51, 'فعالیت': 52, 'حسگر': 53, 'موضوع': 54, 'الزام': 55, 'توصیف_گر': 56, 'كنش_پذیر': 57, 'پدیده': 58, 'محصول': 59, 'مالک': 60, 'زمان': 61, 'جستجو_کننده': 62, 'موجودیت_خلق_شده': 63, 'شخص': 64, 'وضعیت_نهایی': 65, 'اصل_و_نسب': 6

In [ ]:
df['frameNetTags'] = df['frameNetTags'].apply(lambda label : [frame2id[l] for l in label])

<ipython-input-27-b2dc9e216140>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['frameNetTags'] = df['frameNetTags'].apply(lambda label : [frame2id[l] for l in label])


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["Words"], examples["propBankTags"], truncation=True, padding=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[f"frameNetTags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)
tokenized_data = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
tokenized_data = tokenized_data.train_test_split(test_size=0.3)

In [ ]:
import evaluate
seqeval = evaluate.load("seqeval")

In [ ]:
accuracy = evaluate.load("accuracy")

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained("myrkur/Persian-ModernBert-base", num_labels=len(frame2id)+1,ignore_mismatched_sizes=True)

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/299M [00:00<?, ?B/s]

In [ ]:
label2id = dict(frame2id)

In [ ]:
id2label = dict((v,k) for k,v in label2id.items())

In [ ]:
label_list = list(frame2id.keys())
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.metrics import classification_report
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels_bin = list(np.concatenate(true_labels))
    pred_labels_bin = list(np.concatenate(true_predictions))
    print(true_labels_bin)
    print(pred_labels_bin)
    report = classification_report(true_labels_bin, pred_labels_bin)
    metrics = precision_recall_fscore_support(true_labels_bin, pred_labels_bin, average='weighted')
    accuracy = metrics[0]
    precision = metrics[1]
    recall = metrics[2]
    f1_score = metrics[3]
    return {
        "precision": precision,
        "recall": recall,
        "f1": f1_score,
        "accuracy": accuracy
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="bert-token-classification",
    learning_rate=0.1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    evaluation_strategy="epoch",
    weight_decay=0.01,
    save_strategy="no",
    #use_cpu=True,
    #no_cuda=True,
    #load_best_model_at_end=True, # uncomment if you wanna save
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(model=model,args=training_args,data_collator=data_collator,train_dataset=tokenized_data['train']  ,eval_dataset=tokenized_data['test'],compute_metrics=compute_metrics,)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()